In [2]:
import MeCab as mc
import re
import pandas as pd
import numpy as np
import urllib.request
import unicodedata
from sklearn.model_selection import train_test_split

In [23]:
#retweet dataの読み込み
retweet_input = pd.read_csv(filepath_or_buffer="t_retweet.csv", encoding="utf_8", sep=",")
print(len(retweet_input))

#tweet dataの読み込み
tweet_input = pd.read_csv(filepath_or_buffer="t_tweet_data.csv", encoding="utf_8", sep=",")
print(len(tweet_input))



#retweetしたデータとしなかったデータに分割
retweet = retweet_input.query('retweet == 1')
n_retweet = retweet_input.query('retweet == 0')

#retweetしたデータ
r_data = pd.merge(retweet, tweet_input, how="inner", on="tweetID")
#retweetしなかったデータ
nr_data = pd.merge(n_retweet, tweet_input, how="inner", on="tweetID")

#余分に別れたカラムの結合
r_data['tweet2'] = r_data[['tweet', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2],x[3],x[4]), axis=1)
nr_data['tweet2'] = nr_data[['tweet', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2],x[3],x[4]), axis=1)

#必要なカラム以外は取り除く
r_data.drop(['date_y','tweet','Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)
nr_data.drop(['date_y','tweet','Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

#不要な文字の削除
r_data["tweet2"] = r_data["tweet2"].str.replace('\n', ' ')
r_data["tweet2"] = r_data["tweet2"].str.replace('_nan', ' ')
nr_data["tweet2"] =nr_data["tweet2"].str.replace('\n', ' ')
nr_data["tweet2"] = nr_data["tweet2"].str.replace('_nan', ' ')

print(r_data.head(3))
print(nr_data.head(3))

48401
48872
        tweetID  retweet               date_x  \
0  7.113431e+17        1  2016-03-20 00:00:00   
1  7.124711e+17        1  2016-03-23 00:00:00   
2  7.223425e+17        1  2016-04-19 00:00:00   

                                              tweet2  
0                         本日も遠野風の丘軒下市、よろしくお願いします。     
1  【遠野山ぶどうワイン2015】本日より発売開始です！！駅前売店でも入荷しております♪数量限定...  
2  骨寺荘園室より 【世界遺産 拡張資産 巡回バス】のお知らせ  ４月23日（土）に世界遺産拡張...  
        tweetID  retweet               date_x  \
0  7.111280e+17        0  2016-03-19 00:00:00   
1  7.113160e+17        0  2016-03-20 00:00:00   
2  7.113729e+17        0  2016-03-20 00:00:00   

                                              tweet2  
0  きょうは来てくれてありがとっ♡ お買い物してくれてありがとっ♡ 21日は陸前高田にねば〜るく...  
1  みんなゆめ☆も〜にん☆彡  かさ上げに 思いはひとつ 笑顔かなっ♪            ゆ...  
2  【遠野風の丘軒下市】ラム肉まんが通常220円のところ200円、2つで300円で販売しています...  


In [28]:
#データをランダムに5000件ずつ抽出
ran_r_data = r_data.sample(n=5000)
ran_nr_data = nr_data.sample(n=5000)
#データの結合
data = ran_r_data.append(ran_nr_data)
#dateでソート
data_s = data.sort_values('date_x')
print(data_s.head(10))

In [43]:
data_s.to_csv("retweet_data.txt",index=False, sep=",")

In [85]:
#テキストデータのみ取り出し、ファイルに書き出す
text = data_s.drop(['tweetID', 'retweet', 'date_x'], axis=1)
text.to_csv("tweet.txt",header=False, index=False, sep=",")
#retweetのラベルのみ取り出す
tweet_labe = data_s.drop(['tweetID', 'date_x', 'tweet2'], axis=1)
tweet_labe.to_csv("label.txt", header=Falsem index=False, sep=",")

In [51]:
#ツイートのテキスト読み込み
test = open("tweet.txt", "r")
lines = test.readlines()
test.close()
print(len(lines))

10000


In [4]:
#テキストのクリーニング
#text_data_test  = lines[9924:9929]
#print(text_data_test)
def clean_text(text_string):
    text_string = re.sub(r'([^\s\w])+', '', text_string)
    text_string = re.sub(r'[0-9]', '', text_string)
    text_string = " ".join(text_string.split())
    text_string = text_string.lower()
    return(text_string)

#text_data_train = [clean_text(x) for x in text_data_test]
#print(text_data_train)

In [5]:
#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

In [6]:
stopwords = get_stopword()
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', 'ハイ', '上', '中', '下', '字', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', '段', '

In [7]:
#MeCabによる形態素解析
def extractter(text):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if nes[0] == u"名詞":
                if nes[6] == u"*":
                     keyword.append(wor)
                else:
                    keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

In [8]:
text = '【祝】北海道新幹線が開業 | 2016年3月26日(土) -    北海道の皆さん、全国の皆さん、新幹線で岩手におでんせ～♪あっ、飛行機もアルヨ‼'
text = clean_text(text)
result = extractter(text)
print(result)

['祝', '北海道', '新幹線', '開業', '年月', '日土', '北海道', '皆さん', '全国', '皆さん', '新幹線', '岩手', 'おでん', '飛行機', 'アルヨ']


In [141]:
#ストップワードの除去
def except_stopwords(text, stopwords):
    for i in text:
        for j in stopwords:
            if i == j:
                text.remove(j)
    return text

In [142]:
s_result = except_stopwords(result, stopwords)
print(s_result)

['祝', '北海道新幹線', '開業', '年月', '日土', '北海道', '皆さん', '全国', '皆さん', '新幹線', '岩手', 'おでん', 'する', '飛行機', 'アル']


In [ ]:
#データセットの分割
x_train, x_test, y_train, y_test = train_test_split(namelist2_x, namelist2_y, test_size=0.3)